In [14]:
import pandas as pd
from pdfminer.high_level import extract_text
import glob
import re
from collections import defaultdict

In [3]:
path = '/Users/thomaslee/Desktop/Community Board/resolutions/*.pdf'
files = glob.glob(path)

In [ ]:
data = []
for file in files:
    try:
        # Extract text from each PDF file
        text = extract_text(file)

        # Append a tuple of filename and text to the list
        data.append((file, text))
    except Exception as e:
        print(f"Error processing {file}: {e}")

# Create a DataFrame from the collected data
df = pd.DataFrame(data, columns=['filename', 'text'])

In [ ]:
# Save the DataFrame to a CSV file
df.to_csv('resolutions.csv', index=False)

In [4]:
# Open the CSV file as a DataFrame
df = pd.read_csv('voting_result.csv')
df.head()

,filename,text
0,/content/drive/MyDrive/community_board_data_ha...,COMMUNITY BOARD #1 – MANHATTAN\nRESOLUTION\n\n...
1,/content/drive/MyDrive/community_board_data_ha...,COMMUNITY BOARD 1 – MANHATTAN \n\nRESOLUTION \...
2,/content/drive/MyDrive/community_board_data_ha...,COMMUNITY BOARD 1 – MANHATTAN \nRESOLUTION \n\...
3,/content/drive/MyDrive/community_board_data_ha...,COMMUNITY BOARD 1 – MANHATTAN \nRESOLUTION \n\...
4,/content/drive/MyDrive/community_board_data_ha...,COMMUNITY BOARD 1 – MANHATTAN \nRESOLUTION \n\...


In [23]:
# Print the text of the first PDF file
print(df['text'][8])

COMMUNITY BOARD 1 – MANHATTAN 
RESOLUTION 

DATE: JULY 26, 2022 

COMMITTEE OF ORIGIN: ENVIRONMENTAL PROTECTION 

COMMITTEE VOTE: 
PUBLIC VOTE: 
BOARD VOTE: 

9 In Favor 
0 In Favor 
 41 In Favor 

1 Opposed 
0 Opposed 
0 Opposed 

0 Abstained 
0 Abstained 
0 Abstained 

0 Recused  
0 Recused 
0 Recused 

RE: 

Manhattan Community Board 1 Review of the Lower Manhattan Coastal 
Resiliency Battery Project for the Public Design Commission 

WHEREAS:  Manhattan Community Board 1’s (CB 1) Environmental Protection Committee 

convened a public meeting to hear a presentation of the Lower Manhattan Coastal 
Resiliency (LMCR) proposal by the Mayor’s Office of Climate & Environmental 
Justice (MOCEJ) the Department of Parks & Recreation (DPR), and the Stantec 
Corporation on July 18th, 2022; and 

WHEREAS:  The design review was largely accepted by the committee without comment, save 

for the area that would be reconfigured for DPR storage in service of parks 
maintenance within the district th

In [12]:
def extract_info(text):
    # Extract the date
    date_pattern = r"DATE:\s*([A-Z]+\s*\d+,\s*\d{4})"
    dates = re.findall(date_pattern, text)
    date = dates[0] if dates else "Date not found"  # Assuming the date is the same throughout the document

    # Extract committees
    committee_pattern = r"COMMITTEE OF ORIGIN:\s*([^\n]+)"
    committees = re.findall(committee_pattern, text)

    # Extract topics
    topic_pattern = r"RE:\n\n([^\n]+)"
    topics = re.findall(topic_pattern, text)

    # Extract resolutions
    resolution_pattern = r"THEREFORE\nBE IT\nRESOLVED\nTHAT:\n\n([^\n]+)"
    resolutions = re.findall(resolution_pattern, text)

    return {
        "Date": date,
        "Committees": committees,
        "Topics": topics,
        "Resolutions": resolutions
    }

In [24]:
# Parse the text of the first PDF file
parsed_data = extract_info(df['text'][0])

# Print the parsed data
parsed_data

{'Date': 'FEBRUARY 5, 2015',
 'Committees': ['LANDMARKS',
  'LANDMARKS',
  'LANDMARKS',
  'LANDMARKS',
  'PLANNING',
  'PLANNING',
  'PLANNING',
  'QUALITY OF LIFE',
  'QUALITY OF LIFE',
  'QUALITY OF LIFE',
  'SEAPORT/CIVIC CENTER',
  'SPECIAL LANDMARKS',
  'TRIBECA',
  'TRIBECA',
  'TRIBECA',
  'TRIBECA',
  'BATTERY PARK CITY',
  'FINANCIAL DISTRICT',
  'FINANCIAL DISTRICT',
  'FINANCIAL DISTRICT',
  'FINANCIAL DISTRICT'],
 'Topics': ['79 Laight Street, application for replacement of entrance door and transom',
  '152 Franklin Street, application for handicapped access ramp',
  '140 Broadway, application for entry infill and re-glazing',
  '464 Greenwich Street, amendment to storefront glazing',
  'N 150167ZRY',
  'N 150189PXM',
  'Proposed Action Plan Amendment 8 and NYC EDC’s reallocation of',
  'Illegal Hotels',
  'Resolution in opposition to Intro 585 which would amend the City Charter to',
  'James Zadroga 9/11 Health and Compensation Reauthorization Act',
  '87 South Street, ap

In [15]:
# Initialize a dictionary to hold the data
data = defaultdict(lambda: defaultdict(int))

In [16]:
# Loop over each text entry in the DataFrame
for index, row in df.iterrows():
    parsed_data = extract_info(row['text'])  # Assuming 'text' is the column with your data
    date = parsed_data['Date']
    for committee in parsed_data['Committees']:
        data[date][committee] += 1

In [17]:
# Convert the dictionary to a DataFrame
committee_data = pd.DataFrame(data).T.fillna(0).astype(int)
committee_data.index.name = 'Date'
committee_data.reset_index(inplace=True)

In [19]:
committee_data

,Date,LANDMARKS,PLANNING,QUALITY OF LIFE,SEAPORT/CIVIC CENTER,SPECIAL LANDMARKS,TRIBECA,BATTERY PARK CITY,FINANCIAL DISTRICT,ENVIRONMENTAL PROTECTION,...,SEAPORT / CIVIC CENTER,TRIBECA LICENSING TASK FORCE,TRIBECA LICENSING TASK FORCE,TRIBECA LICENSING COMMITTEE,TRIBECA COMMITTEE,INTERNAL WORKINGS OF THE,SMALL BUSINESS TASK FORCE,INTERNAL WORKINGS TASK FORCE,ART & ENTERTAINMENT,WTC/REDEVELOPMENT
0,"FEBRUARY 5, 2015",4,3,3,1,1,4,1,4,0,...,0,0,0,0,0,0,0,0,0,0
1,"JANUARY 25, 2022",0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
2,"FEBRUARY 22, 2022",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,"NOVEMBER 22, 2022",0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
4,"MARCH 22, 2022",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
227,"JULY 29, 2003",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,2
228,"SEPTEMBER 16, 2003",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
229,"NOVEMBER 18, 2003",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
230,"OCTOBER 21, 2003",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [27]:
# Check the data types of Date column
committee_data['Date'].dtype

# Convert the 'Date' column to datetime format
committee_data['Date'] = pd.to_datetime(committee_data['Date'])

# Sort the DataFrame by the 'Date' column from latest to oldest
committee_data.sort_values('Date', ascending=False, inplace=True)

committee_data.head()

/var/folders/1r/2wpzk2cd1ln3_nc94hvh0cqm0000gn/T/ipykernel_56546/2571670657.py:5: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  committee_data['Date'] = pd.to_datetime(committee_data['Date'])


,Date,LANDMARKS,PLANNING,QUALITY OF LIFE,SEAPORT/CIVIC CENTER,SPECIAL LANDMARKS,TRIBECA,BATTERY PARK CITY,FINANCIAL DISTRICT,ENVIRONMENTAL PROTECTION,...,SEAPORT / CIVIC CENTER,TRIBECA LICENSING TASK FORCE,TRIBECA LICENSING TASK FORCE,TRIBECA LICENSING COMMITTEE,TRIBECA COMMITTEE,INTERNAL WORKINGS OF THE,SMALL BUSINESS TASK FORCE,INTERNAL WORKINGS TASK FORCE,ART & ENTERTAINMENT,WTC/REDEVELOPMENT
24,2024-03-26,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
23,2024-02-27,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
22,2024-01-23,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
35,2023-12-20,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
34,2023-11-28,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [20]:
committee_data.to_csv('committee_data.csv', index=False)